# Interactive visualization of Holland Model parameters

In [19]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd
import bqplot as bq

In [20]:
from bqplot import *

In [21]:
from bqplot import pyplot as plt

In [22]:
from traitlets import dlink

In [23]:
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)

In [24]:
from ipywidgets import Label, ToggleButtons, VBox

In [25]:

# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [26]:
wind_radii=np.array([34.,50.,64.])*kt2ms

In [27]:
def hvel(r,dic):
    for key,val in dic.items():
        exec(key + '=val') 
    x=0.5+(r-rmax)/(maxR-rmax)*k
    return (b/rhoa*(rmax/r)**b*dph*np.exp(-(rmax/r)**b))**x

In [28]:
rh=np.linspace(0.01,maxR,100)

### read TC data

In [29]:
tc=pd.read_csv('../tmp/step3.txt',delimiter='\t')

In [30]:
tc

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w34sw,w34nw,vmax0vt,b,dph,k,np.max(V),rmaxh,rmse,vmax1
0,0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,...,19.534445,20.608431,26.161934,0.824513,2687.041711,0.098787,26.161934,26013.373175,3.003832,26.621949
1,1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,...,19.263473,20.915893,26.058509,0.849102,2575.365636,0.134372,26.058509,21960.064705,2.634959,26.448632
2,2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,...,19.538575,20.351526,30.315362,1.161375,2532.480755,0.239280,30.315362,19995.524368,2.827389,30.673483
3,3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,...,19.796023,20.766386,32.334880,1.044077,3184.888911,0.207510,32.334880,15661.528632,3.298494,32.614984
4,4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,...,20.326892,20.615410,32.122624,1.064447,3095.902038,0.184050,32.122624,19347.964022,3.666758,32.468297
5,5.0,2016-09-29 17:00:00,MATTHEW,14.3,-66.4,992,-99,68524.0,31.381111,253724.0,...,20.763431,20.727831,26.362415,0.802108,2809.292402,0.082869,26.362415,27162.529686,1.590054,26.848414
6,6.0,2016-09-29 18:00:00,MATTHEW,14.4,-66.6,990,-99,72228.0,31.381111,251872.0,...,20.189763,20.559208,26.567329,0.828845,2737.803855,0.127165,26.567329,20852.178809,2.715117,26.942734
7,7.0,2016-09-29 19:00:00,MATTHEW,14.4,-66.9,990,-99,51856.0,31.381111,257428.0,...,20.409975,19.850360,26.408986,0.832996,2704.446141,0.098270,26.408986,24241.126370,2.599534,26.845065
8,8.0,2016-09-29 20:00:00,MATTHEW,14.5,-67.1,989,-99,53708.0,32.410000,250020.0,...,20.292461,19.585819,26.301417,0.809829,2746.933958,0.119317,26.301417,20617.337892,2.634820,26.676247
9,9.0,2016-09-29 21:00:00,MATTHEW,14.6,-67.3,989,-99,44448.0,32.924444,238908.0,...,20.423490,18.014881,32.730365,1.050738,3237.975920,0.217043,32.730365,14243.179768,3.317426,32.990424


In [31]:
i=10

In [32]:
tc.ix[i:i,['dph','b','rmax','k']].T.to_dict()[i]


{'b': 1.10770605328,
 'dph': 2992.5566233099998,
 'k': 0.223801641105,
 'rmax': 42596.0}

In [34]:
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

ax_x = bq.Axis(label='X Axis', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y Axis', scale=y_sc, orientation='vertical', grid_lines='solid')

prop = tc.ix[i:i,['dph','b','rmax','k']].T.to_dict()[i]

p64=tc.iloc[i][['64ne','64se', '64sw', '64nw']].astype(float)

p50=tc.iloc[i][['50ne','50se', '50sw', '50nw']].astype(float)

p34=tc.iloc[i][['34ne','34se', '34sw', '34nw']].astype(float)

vmax=tc.iloc[i].vmax

x_data=np.concatenate([p34.values,p50.values,p64.values])

x_data[x_data==0.]=np.nan

y_data=np.concatenate([np.ones(4)*wind_radii[0],np.ones(4)*wind_radii[1],np.ones(4)*wind_radii[2]])

y_data[np.isnan(x_data)]=np.nan

y_data=y_data[~np.isnan(y_data)]

x_data=x_data[~np.isnan(x_data)]


Line = bq.Lines(x=rh,
                     y=hvel(rh, prop),
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc})


LineVmax = bq.Lines(x=rh,
                     y=np.ones(rh.size)*vmax,
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc},colors=['black'], line_style = 'dashed' )



scatter_chart = Scatter(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc},default_colors=['red'])
scatter_chart.enable_move=True


slider1 = IntSlider(value=0, min=0 , max=tc.shape[0]-1 , step=1, description='time')

# Wiring the function parameter with a slider
def select_time(change):
    new = change['new']
    prop = tc.ix[new:new,['dph','b','rmax','k']].T.to_dict()[new]
    p64=tc.iloc[new][['64ne','64se', '64sw', '64nw']].astype(float)
    p50=tc.iloc[new][['50ne','50se', '50sw', '50nw']].astype(float)
    p34=tc.iloc[new][['34ne','34se', '34sw', '34nw']].astype(float)
    vmax=tc.iloc[new].vmax
    x_data=np.concatenate([p34.values,p50.values,p64.values])
    x_data[x_data==0.]=np.nan
    y_data=np.concatenate([np.ones(4)*wind_radii[0],np.ones(4)*wind_radii[1],np.ones(4)*wind_radii[2]])
    y_data[np.isnan(x_data)]=np.nan
    y_data=y_data[~np.isnan(y_data)]
    x_data=x_data[~np.isnan(x_data)]
    Line.y = hvel(rh, prop)
    LineVmax.y = np.ones(rh.size)*vmax
    scatter_chart.x = x_data
    scatter_chart.y = y_data


select_time({'new': i})
slider1.observe(select_time, names=['value'])


fig = bq.Figure(axes=[ax_x, ax_y], marks=[Line, scatter_chart, LineVmax], title_style={'font-size': '20px','fill': 'DarkOrange'}, title='')


#compute rmse
#rmse=hvel(x_data, slider1.value, slider2.value, slider3.value, slider4.value)-y_data
#rmse_label.text = [str(rmse.sum())]
    

VBox([fig, slider1])



In [ ]:
tc.ix[1:1,['deltap','b','rmax','k']].T.to_dict()[1]

In [81]:
tc.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w34sw,w34nw,vmax0vt,b,dph,k,np.max(V),rmaxh,rmse,vmax1
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.000000,0.000000,5.069814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,18.059481,16.718113,18.059481,0.814397,1288.689225,0.235803,18.059481,21236.795240,0.832487,18.322971
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,18.366306,17.301589,18.366306,0.844967,1291.038196,0.238350,18.366306,22989.998120,0.705101,18.680701
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,18.525193,17.452947,25.859722,1.076894,1981.089735,0.234723,25.859722,17570.521694,2.037823,26.124160
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,18.632523,17.383112,25.728973,1.158931,1829.735742,0.223864,25.728973,19772.107096,1.984706,26.045152
